In [1]:
import numpy as np
import pandas as pd
import random
import time
import pickle

In [2]:
with open("data.txt","r") as fp:
    lines=fp.readlines()

In [3]:
d={}
columns=["building","pic"]
for i in range(128):
    columns.append("x_"+str(i))
for col in columns:
    d[col]=[]
for line in lines:
    line=line.split(" ")[1].split("\\")
    d["building"].append(line[0])
    line=line[1].split(".JPG:")
    d["pic"].append(line[0])
    line=line[1].split(",")
    for i in range(128):
        d["x_"+str(i)].append(int(float(line[i])))
data=pd.DataFrame(d)

In [11]:
print(data.shape[0])

122514


In [12]:
def euclid2(p1,p2): 
    p=p1-p2
    p=p**2
    return np.sum(p)

In [13]:
def euclid(p1,p2): 
    return sum([(p1[i]-p2[i])**2 for i in range(len(p1))])

In [14]:
def findCentroid(centroids,point):
    minDist = -1
    pos = 0
    for i in range(len(centroids)):
        dist = euclid2(point[0],centroids[i]["coordinates"])
        if minDist == -1:
            minDist = dist
            pos = i
        elif minDist > dist:
            minDist = dist
            pos = i
    return pos

In [15]:
def initializeCentroids2(centroids,points):
    for i in range(len(points)):
        center=findCentroid(centroids,points[i])
        points[i][1]=center
        centroids[center]["points"][i]=True
        centroids[center]["num"]+=1

In [16]:
def initializeCentroids(centroids,points):
    for i in range(len(points)):
        center=findCentroid(centroids,points[i])
        points[i][1]=center
        centroids[center]["points"].append(i)

In [17]:
def updateCentroids2(centroids,points):
    for centroid in centroids:
        arr=[]
        for i in range(len(points)):
            if(centroid["points"][i]):
                arr.append(points[i][0])
        if (len(arr)==0):
            centroid["coordinates"]=np.zeros(128)
        else: 
            centroid["coordinates"]=np.mean(arr,axis=0)

In [18]:
def updateCentroids(centroids,points):
    for centroid in centroids:
        arr=[]
        for point in centroid["points"]:
            arr.append(points[point])
        if (len(arr)==0):
            centroid["coordinates"]=np.zeros(128)
        else: 
            centroid["coordinates"]=np.mean(arr,axis=0)

In [19]:
def updatePoints2(centroids,points):
    updated=False
    done=[False]*len(points)
    for centroid in centroids:
        for i in range(len(points)):
            if(centroid["points"][i]):
                if(not done[centroid["points"][i]]):
                    done[centroid["points"][i]]=True
                    center=findCentroid(centroids,points[centroid["points"][i]])
                    if (center != points[i][1]):
                        updated=True
                        points[centroid["points"][i]][1]=center
                        centroids[center]["points"][i]=True
                        centroids[center]["num"]+=1
                        centroid["points"][i]=False
                        centroid["num"]+=-1
    return updated

In [20]:
def updatePoints(centroids,points):
    updated=False
    done=[False]*len(points)
    for centroid in centroids:
        for i in range(len(centroid["points"])-1,-1,-1):
            if(not done[centroid["points"][i]]):
                done[centroid["points"][i]]=True
                center=findCentroid(centroids,points[centroid["points"][i]])
                if (center != points[centroid["points"][i]][1]):
                    updated=True
                    points[centroid["points"][i]][1]=center
                    centroids[center]["points"].append(centroid["points"][i])
                    del centroid["points"][i]
    return updated

In [21]:
def cluster(k,pic):
    features=data.loc[data['pic'] == pic]
    feat = features[features.columns.difference(['building', 'pic'])].values.tolist()
    points=[]
    for f in feat:
        points.append([f,-1])
    centers=random.sample(feat,k)
    centroids=[{"coordinates":center , "points":[]} for center in centers]
    initializeCentroids(centroids,points)
    while (True):
        updateCentroids(centroids,points)
        if(not updatePoints(centroids,points)):
            break
    return centroids      

In [22]:
def cluster2(k):
    feat = data[data.columns.difference(['building', 'pic'])].values.tolist()
    points=[]
    for f in feat:
        points.append([np.asarray(f),-1])
    centers=random.sample(feat,k)
    centroids=[{"coordinates":np.asarray(center) , "points":[False for i in range(len(points))] , "num":0} for center in centers]
    initializeCentroids2(centroids,points)
    while (True):
        updateCentroids2(centroids,points)
        if(not updatePoints2(centroids,points)):
            break
    return centroids

In [24]:
start=time.time()
a=cluster(2)
end  = time.time()
print(end-start)

ValueError: operands could not be broadcast together with shapes (128,) (2,) 

In [10]:
al=[]
for i in range(250,800,50):
    start=time.time()
    a=cluster2(i)
    end=time.time()
    print(end-start)
    al.append(a)
    pickle.dump(a, open(("cluster"+str(i)+".p"), "wb" ))

KeyboardInterrupt: 

In [40]:
def clusterAll(kcluster,kn,testPics):
    #pictures=[['1', 'DSC05903'], ['1', 'DSC05904'], ['1', 'DSC05905'], ['1', 'DSC05906'], ['1', 'DSC05907'], ['2', 'DSC05908'], ['2', 'DSC05909'], ['2', 'DSC05910'], ['2', 'DSC05911'], ['2', 'DSC05912'], ['3', 'DSC05913'], ['3', 'DSC05914'], ['3', 'DSC05915'], ['3', 'DSC05916'], ['4', 'DSC05917'], ['4', 'DSC05918'], ['4', 'DSC05919'], ['4', 'DSC05920'], ['4', 'DSC05921'], ['5', 'DSC05922'], ['5', 'DSC05923'], ['5', 'DSC05924'], ['5', 'DSC05925'], ['5', 'DSC05926'], ['5', 'DSC05927'], ['5', 'DSC05928'], ['6', 'DSC05929'], ['6', 'DSC05930'], ['6', 'DSC05931'], ['6', 'DSC05932'], ['6', 'DSC05933'], ['7', 'DSC05934'], ['7', 'DSC05935'], ['7', 'DSC05936'], ['7', 'DSC05937'], ['7', 'DSC05938'], ['7', 'DSC05939'], ['8', 'DSC05940'], ['8', 'DSC05941'], ['8', 'DSC05942'], ['8', 'DSC05943'], ['8', 'DSC05944'], ['8', 'DSC05945'], ['8', 'DSC05946'], ['8', 'DSC05947'], ['8', 'DSC05948'], ['8', 'DSC05949'], ['8', 'DSC05950'], ['8', 'DSC05951']]
    pictures2=[['DSC05903','DSC05904', 'DSC05905', 'DSC05906','DSC05907'],['DSC05908','DSC05909', 'DSC05910','DSC05911','DSC05912'], ['DSC05913', 'DSC05914', 'DSC05915', 'DSC05916'], ['DSC05917','DSC05918', 'DSC05919','DSC05920','DSC05921'], ['DSC05922', 'DSC05923', 'DSC05924', 'DSC05925', 'DSC05926','DSC05927','DSC05928'], ['DSC05929','DSC05930','DSC05931', 'DSC05932', 'DSC05933'], ['DSC05934', 'DSC05935', 'DSC05936', 'DSC05937','DSC05938','DSC05939'], ['DSC05940', 'DSC05941','DSC05942', 'DSC05943', 'DSC05944', 'DSC05945', 'DSC05946','DSC05947', 'DSC05948', 'DSC05949','DSC05950','DSC05951']]
    test={}
    train={}
    columns=["building","pic"]     
    for i in range(kcluster):
        columns.append("x_"+str(i))
    for col in columns:
        test[col]=[]
        train[col]=[]
    for i in range(len(pictures2)):
        random.shuffle(pictures2[i])
        j=0
        n=0
        while(n<kn):
            if([str(i+1),pictures2[i][j]] not in testPics):
                clus=cluster(kcluster,pictures2[i][j])
                n=n+1
                train["building"].append(str(i+1))
                train["pic"].append(pictures2[i][j])
                for k in range(kcluster):
                    train["x_"+str(k)].append(clus[k])
            j=j+1
    for pic in testPics:
        clus=cluster(kcluster,pic[1])
        test["building"].append(pic[0])
        test["pic"].append(pic[1])
        for i in range(kcluster):
            test["x_"+str(i)].append(clus[i])
    return [pd.DataFrame(train),pd.DataFrame(test)]

In [41]:
c=clusterAll(2,1,[['3', 'DSC05915'],['1', 'DSC05904']])
print(c)

[  building       pic   x_0   x_1
0        1  DSC05903  1792   708
1        2  DSC05908   471  2029
2        3  DSC05913   718  1782
3        4  DSC05920  1709   791
4        5  DSC05928   455  2046
5        6  DSC05931  1605   895
6        7  DSC05935  1844   656
7        8  DSC05950  2182   319,   building       pic   x_0  x_1
0        3  DSC05915  2076  424
1        1  DSC05904  1603  897]


In [16]:
def euclidr(row1,row2,clusterSize): 
    return [(sum([(row1["x_"+str(i)]-row2["x_"+str(i)])**2 for i in range(clusterSize)])),row2['building']]

In [56]:
def knn(k,train,test,clusterSize):
    pred = []
    distances=[]
    for i in range(test.shape[0]):
        distances.append([])
    for i in range(train.shape[0]):
        dist=test.apply(euclidr, axis=1, args=(train.iloc[i],clusterSize)).tolist()
        for i in range(len(dist)):
            distances[i].append(dist[i])
    closest=[]
    for d in distances:
        d.sort(key = lambda x: x[0])
        v=[0]*8
        for di in d:
            if (v[int(di[1])-1]==k-1):
                closest.append(di[1])
                break
            v[int(di[1])]=v[int(di[1])]+1
    test["closest"]=closest
    return(test)

In [58]:
print(knn(1,c[0],c[1],2))

  building       pic   x_0  x_1 closest
0        3  DSC05915  2076  424       8
1        1  DSC05904  1603  897       6
